# 🦠 COVID Atlas Dashboard

An interactive visualization of COVID-19 cases, deaths, and life expectancy trends using Dash and Plotly.

## 1. 📚 Import Libraries

In [18]:
import pandas as pd
import plotly.express as px
import pandas as pd
import numpy as np
import re
from jupyter_dash import JupyterDash
import folium
from folium.plugins import HeatMap, HeatMapWithTime
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
from dash import html, dcc, Input, Output
import pandas as pd
import datetime
# import dash
from dash import html, Dash
pd.set_option('display.max_rows', 100)

## 2. 📂 Load and Preview the Data

In [33]:
df = pd.read_csv('/Users/gp/Desktop/CovidGlobe/data/owid-covid-data.csv') # 'data/owid-covid-data.csv' 

In [37]:
df_countries = pd.read_excel('/Users/gp/Desktop/CovidGlobe/data//Map.xlsx') # 'data/Map.xlsx'

In [39]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN


In [41]:
df = df[~df['location'].isin(['Africa', 'Asia', 'European Union', 'Europe', 'International', 'South America', 'North America','Lower middle income', 'Upper middle income', 'High income','Low income', 'World', 'Oceania', 'North Korea'])]

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215161 entries, 0 to 228982
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    215161 non-null  object 
 1   continent                                   215161 non-null  object 
 2   location                                    215161 non-null  object 
 3   date                                        215161 non-null  object 
 4   total_cases                                 202911 non-null  float64
 5   new_cases                                   202648 non-null  float64
 6   new_cases_smoothed                          201522 non-null  float64
 7   total_deaths                                183912 non-null  float64
 8   new_deaths                                  183683 non-null  float64
 9   new_deaths_smoothed                         182573 non-null  float64
 10  t

## 3. 🧹 Data Preprocessing

In [46]:
df_filtered = df[['continent', 'location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_vaccinations', 'life_expectancy', 'gdp_per_capita', 'extreme_poverty']].copy()

In [48]:
df_filtered.head()

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_vaccinations,life_expectancy,gdp_per_capita,extreme_poverty
0,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,64.83,1803.987,NaN
1,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,64.83,1803.987,NaN
2,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,64.83,1803.987,NaN
3,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,64.83,1803.987,NaN
4,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,64.83,1803.987,NaN


In [50]:
df_filtered = df_filtered[(df_filtered['life_expectancy'].notnull()) & (df_filtered['gdp_per_capita'].notnull()) & (df_filtered['extreme_poverty'].notnull())]

In [52]:
df_filtered['new_cases'] = df_filtered['new_cases'].fillna(0)

In [54]:
df_filtered['year'] = pd.to_datetime(df_filtered['date']).dt.year

In [56]:
df_filtered['month'] = pd.to_datetime(df_filtered['date']).dt.month

In [58]:
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

In [60]:
df_filtered.isnull().sum()

continent                 0
location                  0
date                      0
total_cases             784
new_cases                 0
total_deaths           6669
new_deaths             6819
total_vaccinations    82858
life_expectancy           0
gdp_per_capita            0
extreme_poverty           0
year                      0
month                     0
dtype: int64

In [62]:
df_filtered.head()

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_vaccinations,life_expectancy,gdp_per_capita,extreme_poverty,year,month
1951,Europe,Albania,2020-02-25,NaN,0.0,NaN,NaN,NaN,78.57,11803.431,1.1,2020,2
1952,Europe,Albania,2020-02-26,NaN,0.0,NaN,NaN,NaN,78.57,11803.431,1.1,2020,2
1953,Europe,Albania,2020-02-27,NaN,0.0,NaN,NaN,NaN,78.57,11803.431,1.1,2020,2
1954,Europe,Albania,2020-02-28,NaN,0.0,NaN,NaN,NaN,78.57,11803.431,1.1,2020,2
1955,Europe,Albania,2020-02-29,NaN,0.0,NaN,NaN,NaN,78.57,11803.431,1.1,2020,2


In [64]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118227 entries, 1951 to 228982
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   continent           118227 non-null  object        
 1   location            118227 non-null  object        
 2   date                118227 non-null  datetime64[ns]
 3   total_cases         117443 non-null  float64       
 4   new_cases           118227 non-null  float64       
 5   total_deaths        111558 non-null  float64       
 6   new_deaths          111408 non-null  float64       
 7   total_vaccinations  35369 non-null   float64       
 8   life_expectancy     118227 non-null  float64       
 9   gdp_per_capita      118227 non-null  float64       
 10  extreme_poverty     118227 non-null  float64       
 11  year                118227 non-null  int32         
 12  month               118227 non-null  int32         
dtypes: datetime64[ns](1), float64(8

In [66]:
df_filtered.isnull().sum()

continent                 0
location                  0
date                      0
total_cases             784
new_cases                 0
total_deaths           6669
new_deaths             6819
total_vaccinations    82858
life_expectancy           0
gdp_per_capita            0
extreme_poverty           0
year                      0
month                     0
dtype: int64

In [68]:
df_countries.head()

,latitude,longitude,name
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [70]:
df_dashboard = pd.merge(df, df_countries, how='left', left_on='location', right_on='name')

In [72]:
df_ratio = df_dashboard.groupby('location', as_index=False)[['total_cases', 'total_deaths', 'latitude', 'longitude']].mean()
df_ratio['ratio'] = df_ratio['total_deaths'] / df_ratio['total_cases']*100

In [74]:
df_ratio

,location,total_cases,total_deaths,latitude,longitude,ratio
0,Afghanistan,104147.279381,4587.710191,33.939110,67.709953,4.405022
1,Albania,145229.809623,2077.645702,41.153332,20.168331,1.430592
2,Algeria,149406.101135,4132.145855,28.033886,1.659626,2.765714
3,Andorra,18595.863967,110.946978,42.546245,1.601554,0.596622
4,Angola,48023.704762,1027.933761,-11.202692,17.873887,2.140472
...,...,...,...,...,...,...
229,Wallis and Futuna,398.088798,6.806563,-13.768752,-177.156097,1.709810
230,Western Sahara,NaN,NaN,24.215527,-12.885834,NaN
231,Yemen,6717.758658,1342.035398,15.552727,48.516388,19.977428
232,Zambia,157114.666315,2218.187768,-13.133897,27.849332,1.411827


In [76]:
df_ratio.isnull().sum()

location         0
total_cases     17
total_deaths    21
latitude        26
longitude       26
ratio           21
dtype: int64

In [78]:
df_ratio.dropna()

,location,total_cases,total_deaths,latitude,longitude,ratio
0,Afghanistan,1.041473e+05,4587.710191,33.939110,67.709953,4.405022
1,Albania,1.452298e+05,2077.645702,41.153332,20.168331,1.430592
2,Algeria,1.494061e+05,4132.145855,28.033886,1.659626,2.765714
3,Andorra,1.859586e+04,110.946978,42.546245,1.601554,0.596622
4,Angola,4.802370e+04,1027.933761,-11.202692,17.873887,2.140472
...,...,...,...,...,...,...
227,Vietnam,2.696438e+06,18676.498768,14.058324,108.277199,0.692636
229,Wallis and Futuna,3.980888e+02,6.806563,-13.768752,-177.156097,1.709810
231,Yemen,6.717759e+03,1342.035398,15.552727,48.516388,19.977428
232,Zambia,1.571147e+05,2218.187768,-13.133897,27.849332,1.411827


In [80]:
df_ratio['ratio'] = df_ratio['ratio'].round(2)

In [82]:
df_dashboard.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,latitude,longitude,name
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN,33.93911,67.709953,Afghanistan
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN,33.93911,67.709953,Afghanistan
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN,33.93911,67.709953,Afghanistan
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN,33.93911,67.709953,Afghanistan
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,64.83,0.511,40099462.0,NaN,NaN,NaN,NaN,33.93911,67.709953,Afghanistan


In [84]:
df_final = df_dashboard[df_dashboard['location'] != 'North Korea'].copy()

In [86]:
df_final = df_final[['location', 'date', 'total_cases', 'latitude', 'longitude', 'total_cases_per_million']]

In [88]:
df_final

,location,date,total_cases,latitude,longitude,total_cases_per_million
0,Afghanistan,2020-02-24,5.0,33.939110,67.709953,0.125
1,Afghanistan,2020-02-25,5.0,33.939110,67.709953,0.125
2,Afghanistan,2020-02-26,5.0,33.939110,67.709953,0.125
3,Afghanistan,2020-02-27,5.0,33.939110,67.709953,0.125
4,Afghanistan,2020-02-28,5.0,33.939110,67.709953,0.125
...,...,...,...,...,...,...
215156,Zimbabwe,2022-10-16,257893.0,-19.015438,29.154857,16124.839
215157,Zimbabwe,2022-10-17,257893.0,-19.015438,29.154857,16124.839
215158,Zimbabwe,2022-10-18,257893.0,-19.015438,29.154857,16124.839
215159,Zimbabwe,2022-10-19,257893.0,-19.015438,29.154857,16124.839


In [90]:
df_final.isnull().sum()

location                       0
date                           0
total_cases                12250
latitude                   22710
longitude                  22710
total_cases_per_million    12250
dtype: int64

In [92]:
df_final = df_final.dropna()

In [94]:
df_final

,location,date,total_cases,latitude,longitude,total_cases_per_million
0,Afghanistan,2020-02-24,5.0,33.939110,67.709953,0.125
1,Afghanistan,2020-02-25,5.0,33.939110,67.709953,0.125
2,Afghanistan,2020-02-26,5.0,33.939110,67.709953,0.125
3,Afghanistan,2020-02-27,5.0,33.939110,67.709953,0.125
4,Afghanistan,2020-02-28,5.0,33.939110,67.709953,0.125
...,...,...,...,...,...,...
215156,Zimbabwe,2022-10-16,257893.0,-19.015438,29.154857,16124.839
215157,Zimbabwe,2022-10-17,257893.0,-19.015438,29.154857,16124.839
215158,Zimbabwe,2022-10-18,257893.0,-19.015438,29.154857,16124.839
215159,Zimbabwe,2022-10-19,257893.0,-19.015438,29.154857,16124.839


In [96]:
df_final['year'] = pd.to_datetime(df_final['date']).dt.year
df_final['month'] = pd.to_datetime(df_final['date']).dt.month
df_final

/var/folders/c9/1r105dh53kq10kpr_9b7jgyc0000gn/T/ipykernel_50982/1744809404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['year'] = pd.to_datetime(df_final['date']).dt.year
/var/folders/c9/1r105dh53kq10kpr_9b7jgyc0000gn/T/ipykernel_50982/1744809404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['month'] = pd.to_datetime(df_final['date']).dt.month


,location,date,total_cases,latitude,longitude,total_cases_per_million,year,month
0,Afghanistan,2020-02-24,5.0,33.939110,67.709953,0.125,2020,2
1,Afghanistan,2020-02-25,5.0,33.939110,67.709953,0.125,2020,2
2,Afghanistan,2020-02-26,5.0,33.939110,67.709953,0.125,2020,2
3,Afghanistan,2020-02-27,5.0,33.939110,67.709953,0.125,2020,2
4,Afghanistan,2020-02-28,5.0,33.939110,67.709953,0.125,2020,2
...,...,...,...,...,...,...,...,...
215156,Zimbabwe,2022-10-16,257893.0,-19.015438,29.154857,16124.839,2022,10
215157,Zimbabwe,2022-10-17,257893.0,-19.015438,29.154857,16124.839,2022,10
215158,Zimbabwe,2022-10-18,257893.0,-19.015438,29.154857,16124.839,2022,10
215159,Zimbabwe,2022-10-19,257893.0,-19.015438,29.154857,16124.839,2022,10


In [98]:
df_final = df_final[(df_final['year'] == 2020) & (df_final['month'] <= 6)]

In [100]:
df_final

,location,date,total_cases,latitude,longitude,total_cases_per_million,year,month
0,Afghanistan,2020-02-24,5.0,33.939110,67.709953,0.125,2020,2
1,Afghanistan,2020-02-25,5.0,33.939110,67.709953,0.125,2020,2
2,Afghanistan,2020-02-26,5.0,33.939110,67.709953,0.125,2020,2
3,Afghanistan,2020-02-27,5.0,33.939110,67.709953,0.125,2020,2
4,Afghanistan,2020-02-28,5.0,33.939110,67.709953,0.125,2020,2
...,...,...,...,...,...,...,...,...
214314,Zimbabwe,2020-06-26,561.0,-19.015438,29.154857,35.077,2020,6
214315,Zimbabwe,2020-06-27,567.0,-19.015438,29.154857,35.452,2020,6
214316,Zimbabwe,2020-06-28,567.0,-19.015438,29.154857,35.452,2020,6
214317,Zimbabwe,2020-06-29,574.0,-19.015438,29.154857,35.890,2020,6


In [116]:
df_final_new = df_final.groupby(['location', 'month'], as_index=False)[['latitude', 'longitude', 'total_cases_per_million', 'total_cases']].mean()

## 4. 📊 Dashboard Layout

We define the layout of the Dash app, including:
- Dropdown for continent
- Three visualizations (scatter plot, mortality map, spread map)
- Slider and range selectors

In [121]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

_min = df_final_new['month'].min()
_max = df_final_new['month'].max()

new_cases_min = df_filtered['new_cases'].min()
new_cases_max = df_filtered['new_cases'].max()

app = JupyterDash(external_stylesheets=external_stylesheets)

app.layout = html.Div(style={'backgroundColor': 'white', 'color': '#1E456E', 'textAlign': 'center'}, children =[
    html.H1('Covid-19'),
    html.Br(),
    html.H2('How life expectancy has been affected?'),
    html.H4('Life expectancy influenced by Covid cases'),
    dcc.Graph(id='scatter', figure={},style={'width': '120vh', 'height': '60vh'}),
                value='World',
                options= [{'label': 'World', 'value': 'World'},
                          {'label': 'Asia', 'value': 'Asia'},
                          {'label': 'Europe', 'value': 'Europe'}, 
                          {'label': 'Africa', 'value': 'Africa'},
                          {'label': 'North America', 'value': 'North America'},
                          {'label': 'South America', 'value': 'South America'}, 
                          {'label': 'Oceania', 'value': 'Oceania'}
                         ]),

    html.Br(),
    html.Br(),
    html.H3('Mortality ratio % over the globe'),
    dcc.Graph(id="map", figure={}),
     html.Br(),
     html.Br(),
     dcc.RangeSlider(
        id='range-slider',
        min=1,
        max=19.97,
        step=0.1,
        value=[0, 20],
        marks={
            0: '0',
            5: '5',
            10: '10',
            15: '15',
            20: '20',
            }),
    html.Br(),
    html.Br(),
    html.H3('How COVID-19 spread out over the world'),
    dcc.Graph(id="map2", figure={}),
    dcc.Slider(
        id='time-slider',
        min= 1,
        max= 6 ,
        value= 1,
        marks={1:'1',
               2: '2',
               3: '3',
               4: '4',
               5: '5',
               6: '6'
        }
    )

])

/opt/anaconda3/lib/python3.12/site-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



## 5. 🔄 Callbacks
Each graph is updated dynamically using Dash callbacks:
- Scatter: life expectancy vs cases
- Mortality map: death/case ratio
- Monthly spread map

In [153]:
@app.callback(
    Output('scatter','figure'), 
    Input('dropdown', 'value'))


# Life expectancy scatter plot

def update_graph(location):

    
    if location == 'Asia':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})
        
    elif location == 'Europe':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})
        
    elif location == 'Africa':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})
        
    elif location == 'North America':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})
        
    elif location == 'South America':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})

    elif location == 'Oceania':
        df_plt = df_filtered[df_filtered['continent'] == location]
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})
  
    else:
        df_plt = df_filtered
        df_plt = df_plt.groupby(['year','month']).agg({'date':'min', 'life_expectancy':'mean', 'new_cases':'sum'})

    fig = px.scatter(x=df_plt['date'], y=df_plt['life_expectancy'], size=df_plt['new_cases'], size_max=50,
                     color=df_plt['new_cases'], labels={
                     "x": "Date",
                     "y": "Life Expectancy",
                     "color": "New Cases"
                 })
    
    return fig



@app.callback(
    Output('map', 'figure'),
    [Input('range-slider', 'value')])
def update_month_map(map1): #update_month_map
    v_min = map1[0]
    v_max = map1[1]
    df_cut = df_ratio[(df_ratio['ratio'] >= v_min) & (df_ratio['ratio'] <= v_max)]
    
    base_map = px.scatter_geo(df_cut, 
                              lat = 'latitude',
                              lon = 'longitude',
                              color='ratio',
                              projection='orthographic',
                              opacity = 0.8,
                              hover_name = 'location',
                              hover_data=(['ratio', 'location']),
                              width = 1850,
                              height = 500)
    
    
    return base_map



@app.callback(
        Output('map2', 'figure'),
        #[Input('time-slider', 'start_date'),
     Input('time-slider', 'value'))
def update_ratio_map(map2): #update_ratio_map
    m = map2
    df_slider = df_final_new[df_final_new['month'] == m]
    
    map_fig = px.scatter_geo(df_slider, 
                              lat = 'latitude',
                              lon = 'longitude',
#                               color = 'month',
                              size = 'total_cases_per_million',
                              opacity = 0.8,
                              hover_name = 'location',
                              hover_data=(['total_cases', 'location']),
                              width=1850, height=500,
                              labels={"color": "Months"
                 })
    
    return map_fig
    

## 6. 🚀 Run the App

In [156]:
app.run(mode='inline', port=8051)